In [12]:
#if(!require("knitr")) install.packages("knitr")
if(!require("tree")) install.packages("tree")
if(!require("readr")) install.packages("readr")
#if(!require("knitr")) install.packages("knitr")
if(!require("caret")) install.packages("caret")
#if(!require("gridExtra")) install.packages("gridExtra")
if(!require("randomForest")) install.packages("randomForest")
if(!require("foreach")) install.packages("foreach")
if(!require("e1071")) install.packages("e1071")
if(!require("furrr")) install.packages("furrr")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")
if(!require("gbm")) install.packages("gbm")




Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Loading required package: e1071

Loading required package: gbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘gbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘gridExtra’




In [0]:
require("readr")
require("tidyverse")
#require("knitr")
require("caret")
require("parallel")
require("doParallel")


In [6]:
# data parsing
soldat <- read_csv("/soldat.csv")
soldat$y <- factor(soldat$y,levels = c(-1,1),labels = c("insoluble","soluble"))
n <- nrow(soldat)
p <- ncol(soldat)

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [0]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

In [0]:
v <- cor(soldat %>% select(-y), use = "pairwise.complete.obs")
high_corr <- colnames(soldat)[findCorrelation(v, cutoff = 0.95)]
soldat <- soldat %>% select(-all_of(high_corr))

In [0]:
set.seed(1234)
inTest <- createDataPartition(soldat$y,p = 0.5, list = FALSE)[,1]
test <- soldat[inTest,]
training <- soldat[-inTest,]

In [0]:
control <- trainControl(method = "cv", 
                       number = 5,
                       classProbs = TRUE, 
                       allowParallel = TRUE)
metric <- "Accuracy"

grid_ada <- expand.grid(.interaction.depth = 1, .n.trees = 2000,
 .shrinkage = c(.0001,.001,.01,.1,1,10), .n.minobsinnode = 10)

adaboost_stump <- train(y ~. , data = training, 
                             method = "gbm", 
                             trControl = control,
                             tuneGrid = grid_ada,
                             distribution = "adaboost",
                             metric = metric,
                             verbose = FALSE)
save(adaboost_stump,file="adaboost_stump.Rdata")

In [0]:
# varying the interaction depth
grid_ada <- expand.grid(.interaction.depth = c(1,4,8,16), .n.trees = c(10,100,1000,2000),
 .shrinkage = c(.0001,.001,.01,.1,1), .n.minobsinnode = 10)
adaboost_models <- train(y ~. , data = training, 
                             method = "gbm", 
                             trControl = control,
                             tuneGrid = grid_ada,
                             distribution = "adaboost",
                             metric = metric)
save(adaboost_models,file="bt_models.Rdata")

In [31]:
print(adaboost_stump)

Stochastic Gradient Boosting 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 2252, 2252, 2251, 2253, 2252 
Resampling results across tuning parameters:

  shrinkage  Accuracy   Kappa      
  1e-04      0.6202488  0.000000000
  1e-03      0.7026696  0.295696068
  1e-02      0.7477776  0.437256678
  1e-01      0.7499121  0.455356385
  1e+00      0.7037221  0.365727776
  1e+01      0.4326851  0.001259698

Tuning parameter 'n.trees' was held constant at a value of 2000

Tuning parameter 'interaction.depth' was held constant at a value of 1

Tuning parameter 'n.minobsinnode' was held constant at a value of 10
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were n.trees = 2000, interaction.depth =
 1, shrinkage = 0.1 and n.minobsinnode = 10.


In [32]:
print(adaboost_models)

,shrinkage,interaction.depth,n.minobsinnode,n.trees,Accuracy,Kappa,AccuracySD,KappaSD
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1e-04,1,10,2000,0.6202488,0.000000000,0.0005097366,0.00000000
2,1e-03,1,10,2000,0.7026696,0.295696068,0.0147059835,0.03324836
3,1e-02,1,10,2000,0.7477776,0.437256678,0.0158732701,0.03914242
4,1e-01,1,10,2000,0.7499121,0.455356385,0.0147538739,0.03465800
5,1e+00,1,10,2000,0.7037221,0.365727776,0.0158716565,0.03957523
6,1e+01,1,10,2000,0.4326851,0.001259698,0.1077062321,0.01667590
